![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodal - Save Mask for Inpainting
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>Umgebung einrichten</font> </br></p>
!uv pip install --system --prerelease allow -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

<p><font color='black' size="5">
Maske für Inpainting
</font></p>

Inpainting bezeichnet die Fähigkeit eines KI-Modells (z. B. DALL·E), gezielt Teile eines Bildes zu ergänzen oder zu verändern, basierend auf einer textuellen Beschreibung.

**Funktionsweise:**

**Eingabebild**
Ein Bild mit einem fehlenden oder unerwünschten Bereich (z. B. leere Fläche, beschädigter Teil).

**Maske**
Ein separates Schwarz-Weiß-Bild gleicher Größe:

Weiß (255): Dieser Bereich soll durch KI ergänzt oder ersetzt werden.

Schwarz (0): Dieser Bereich bleibt unverändert.

**Prompt** (Textbeschreibung)
Eine präzise Beschreibung, was in die weißen Maskenbereiche eingefügt werden soll, z.B.:

*Ergänze den Hintergrund mit einer Raumstation und Sternen.*

In [ ]:
# Install
!uv pip install --system --prerelease allow -q gradio

In [ ]:
# Import
import gradio as gr
import numpy as np
import imageio
import threading
import time

from google.colab import files

In [ ]:
# Funktionen
def inpainting(img):
    global output_path, mask_output_path
    # 1. Bild speichern
    output_path = "output_image.png"
    imageio.imwrite(output_path, img["composite"])

    # 2. Maske erstellen
    alpha_channel = img["layers"][0][:, :, 3]
    mask = np.where(alpha_channel == 0, 0, 255).astype(np.uint8)

    # Maske speichern (als Graustufenbild)
    mask_output_path = "mask_image.png"
    imageio.imwrite(mask_output_path, mask)

    # App automatisch schließen
    threading.Thread(target=delayed_shutdown).start()

    return img["background"], mask

def delayed_shutdown():
    time.sleep(1)
    demo.close()
    print("Gradio-App wurde automatisch beendet.")

In [ ]:
# Gradio App
with gr.Blocks() as demo:
    with gr.Row():
        img = gr.ImageMask(
            sources=["upload"],
            layers=False,
            transforms=[],
            format="png",
            label="base image",
            show_label=True
        )
        img1 = gr.Image(label="background image", show_label=True)
        img2 = gr.Image(label="mask image", show_label=True)

    btn_process = gr.Button("Bild verarbeiten und speichern")
    btn_process.click(inpainting, inputs=img, outputs=[img1, img2])

In [ ]:
# App aufrufen
demo.launch()

In [ ]:
files.download(mask_output_path)